In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [2]:
# import input_data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tf_mnist", one_hot=True)

Extracting ./tf_mnist\train-images-idx3-ubyte.gz
Extracting ./tf_mnist\train-labels-idx1-ubyte.gz
Extracting ./tf_mnist\t10k-images-idx3-ubyte.gz
Extracting ./tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-3
training_iters = 100000
batch_size = 128
display_step = 25

In [4]:
n_input = 28
n_steps = 28
n_h1 = 128
n_label = 10

In [5]:
def RNN(x, weights, biases):
    # Prepare data shape to match 'rnn' function requirement
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, num=n_steps, axis=1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_h1, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
# tf Graph input
graph = tf.Graph()
with graph.as_default():
    Xtr = tf.placeholder(tf.float32, [None, n_steps, n_input])
    Ytr = tf.placeholder(tf.float32, [None, n_label])
    
    weights = {
        'out': tf.Variable(tf.random_normal([n_h1, n_label]))
    }
    
    biases = {
        'out': tf.Variable(tf.zeros([n_label]))
    }
    
    pred = RNN(Xtr, weights, biases)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Ytr))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Ytr,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    step  = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        _, loss_, accuracy_ = sess.run([optimizer, cost, accuracy], feed_dict={
            Xtr: batch_x, Ytr: batch_y})
        if step % display_step == 0:
            print ("Iter: %d,  Minibatch Loss: %.4f, Training Accuracy: %.4f"% 
                   (step*batch_size, loss_, accuracy_) )
        step += 1
    print ("Optimization Finished!")
    
    test_len = 128
    test_accuracy = sess.run(accuracy, feed_dict={
        Xtr: mnist.test.images[:test_len].reshape((-1, n_steps, n_input)),
        Ytr: mnist.test.labels[:test_len] })
    print ("Test Accuracy: ", test_accuracy)

Iter: 3200,  Minibatch Loss: 1.3216, Training Accuracy: 0.4766
Iter: 6400,  Minibatch Loss: 0.9152, Training Accuracy: 0.7500
Iter: 9600,  Minibatch Loss: 0.4886, Training Accuracy: 0.8516
Iter: 12800,  Minibatch Loss: 0.5562, Training Accuracy: 0.8125
Iter: 16000,  Minibatch Loss: 0.2787, Training Accuracy: 0.9219
Iter: 19200,  Minibatch Loss: 0.2754, Training Accuracy: 0.9141
Iter: 22400,  Minibatch Loss: 0.2475, Training Accuracy: 0.9062
Iter: 25600,  Minibatch Loss: 0.4024, Training Accuracy: 0.8672
Iter: 28800,  Minibatch Loss: 0.3858, Training Accuracy: 0.8906
Iter: 32000,  Minibatch Loss: 0.2113, Training Accuracy: 0.9531
Iter: 35200,  Minibatch Loss: 0.2019, Training Accuracy: 0.9453
Iter: 38400,  Minibatch Loss: 0.1310, Training Accuracy: 0.9375
Iter: 41600,  Minibatch Loss: 0.0884, Training Accuracy: 0.9844
Iter: 44800,  Minibatch Loss: 0.1334, Training Accuracy: 0.9688
Iter: 48000,  Minibatch Loss: 0.1535, Training Accuracy: 0.9375
Iter: 51200,  Minibatch Loss: 0.2122, Train